# ¿Cuál es la mejor tarifa?

Trabajas como analista para el operador de telecomunicaciones Megaline. La empresa ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. El departamento comercial quiere saber cuál de las tarifas genera más ingresos para poder ajustar el presupuesto de publicidad.

Vas a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos.

# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Paso 1. Descripción de los datos](#data_review)
    * [1 Diccionario de datos](#dictionary)
    * [2 Plan de solucion](#plan1)
    * [3 Conclusiones](#data_review_conclusions)
* [Paso 2. Preprocesamiento de datos](#data_preprocessing)
    * [4 Plan de solucion](#plan2)
    * [5 Valores duplicados](#duplicates)
    * [6 Valores ausentes](#missing)
    * [7 Conclusiones](#data_preprocessing_conclusions)
* [Paso 3. Análisis de los datos](#analisys)
    * [A. Fácil](#easy)
    * [B. Intermedio](#inter)
    * [C. Difícil](#hard)
* [Conclusiones](#end)

# Introducción <a id='intro'></a>

La empresa operadora de Telecomunicaciones MEGALINE ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. Necesitamos saber cuál de los dos planes genera más ingresos para ajustar el presupuesto de publicidad, de acuerdo a los requerimientos del departamento comercial de la compañía.

Vamos a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos. Más adelante, encontrarás en las instrucciones del proyecto cuáles son exactamente los aspectos del comportamiento de los clientes que debes analizar. Determinar qué plan, en promedio, aporta más ingresos es una cuestión que se abordará mediante pruebas estadísticas. Más adelante encontrarás más información al respecto en la sección de instrucciones del proyecto.

Descripción de las tarifas:

Nota: Megaline redondea los segundos a minutos y los megabytes a gigabytes. Para las llamadas, cada llamada individual se redondea: incluso si la llamada duró solo un segundo, se contará como un minuto. Para el tráfico web, las sesiones web individuales no se redondean. En vez de esto, el total del mes se redondea hacia arriba. Si alguien usa 1025 megabytes este mes, se le cobrarán 2 gigabytes.

A continuación, puedes ver una descripción de las tarifas:

Surf

1.	Pago mensual: $20 usd.
2.	500 minutos al mes, 50 SMS y 15 GB de datos.
3.	Si se exceden los límites del paquete:

    o	1 minuto: 3 centavos.

    o	1 SMS: 3 centavos.

    o	1 GB de datos: $10 usd.

Ultimate

1.	Pago mensual: $70 usd.
2.	3000 minutos al mes, 1000 SMS y 30 GB de datos.
3.	Si se exceden los límites del paquete:

    o	1 minuto: 1 centavo.

    o	1 SMS: 1 centavo.
    
    o	1 GB de datos: $7 usd.

[Volver a Contenidos](#back)

[Te proporcionamos algunos comentarios para orientarte mientras completas este proyecto. Pero debes asegurarte de eliminar todos los comentarios entre corchetes antes de entregar tu proyecto.]

[Antes de sumergirte en el análisis de datos, explica por tu propia cuenta el propósito del proyecto y las acciones que planeas realizar.]

[Ten en cuenta que estudiar, modificar y analizar datos es un proceso iterativo. Es normal volver a los pasos anteriores y corregirlos/ampliarlos para permitir nuevos pasos.]

# Paso 1. Descripción de los datos <a id='data_review'></a>

Para este trabajo se tienen los siguientes 5 archivos de datos (`/datasets/megaline_users.csv `, `/datasets/megaline_plans.csv `, `/datasets/megaline_calls.csv `, `/datasets/megaline_internet.csv ` y `/datasets/megaline_messages.csv `) los cuales se leeran con la librería Pandas de Python, utilizando el método de lectura de archivos separados por comas `pd.read_csv()` usando el parámetro sep=';' para leer los datos correctamente. 
De primer instancia, se verificará la información para cada uno de los DataFrames.

## Diccionario de datos <a id='dictionary'></a>

Contamos con cinco Datasets en el conjunto de datos, usaremos todas para hacer el preprocesamiento de los datos y el análisis exploratorio de los datos.
A continuación se muestra el diccionario de datos que enumeroa las columnas de cada uno de los Datasets y se describen los datos que contienen.

- `megaline_users.csv`: cada fila corresponde a un usuario en la Compañia Megaline (datos sobre los usuarios).
    - `'user_id'`: identificador único del usuario.
    - `'first_name'`: nombre del usuario.
    - `'last_name'`: apellido del usuario.
    - `'age'`: edad del usuario (en años).
    - `'reg_date'`: fecha de suscripción (dd, mm, aa).
    - `'churn_date'`: la fecha en la que el usuario dejó de usar el servicio (si el valor es ausente, la tarifa se estaba usando cuando fue extraída esta base de datos).
    - `'city'`: ciudad de residencia del usuario.
    - `'plan'`: nombre de la tarifa ('surf' o 'ultimate')

- `megaline_plans.csv `: cada fila corresponde a... (datos sobre las tarifas).
    - `'plan_name'`: nombre de la tarifa.
    - `'usd_monthly_fee'`: pago mensual en dólares estadounidenses.
    - `'minutes_included'`: minutos incluidos al mes.
    - `'messages_included'`: SMS incluidos al mes.
    - `'mb_per_month_included'`: datos incluidos al mes (en megabytes).
    - `'usd_per_minute'`: precio por minuto tras exceder los límites del paquete (por ejemplo, si el paquete incluye 100 minutos, el operador cobrará el minuto 101).
    - `'usd_per_message'`: precio por SMS tras exceder los límites del paquete.
    - `'usd_per_gb'`: precio por gigabyte de los datos extra tras exceder los límites del paquete (1 GB = 1024 megabytes).

- `megaline_calls.csv`: cada fila corresponde a una llamada única que realizaron los clientes (datos sobre las llamadas).
    - `'id'`: identificador único de la llamada.
    - `'call_date'`: fecha de la llamada.
    - `'duration'`: duración de la llamada (en minutos).
    - `'user_id'`: el identificador del usuario que realiza la llamada.

- `megaline_internet.csv`: cada fila corresponde a una consulta en internet que realizaron los clientes (datos sobre las sesiones web).
    - `'id'`: identificador único de la sesión.
    - `'mb_used'`: el volumen de datos gastados durante la sesión (en megabytes).
    - `'session_date'`: fecha de la sesión web.
    - `'user_id'`: identificador del usuario.

- `megaline_messages.csv`: cada fila corresponde a un mensaje tipo sms que realizaron los clientes (datos sobre los SMS).
    - `'id'`: identificador único del SMS.
    - `'message_date'`: fecha del SMS.
    - `'user_id'`: el identificador del usuario que manda el SMS.

[Volver a Contenidos](#back)

## Plan de solución <a id='plan1'></a>

Descripción de los datos.

Para iniciar con la revisión de todos los Datasets, vamos a importa las librerias de Pandas, Numpy y Stats de Scipy que nos serviran para crear un Dataframe por cada uno de los archivos csv.

De esta manera será posible revisar la información contenida en cada archivo, realizando el Preprocesamiento de los datos; como corrección de los nombres de las columnas para cumplir con el formato SnakeCase; rellenar valores ausentes, y eliminar datos duplicados de ser necesario.

Finalmente, realizaremos el Análisis de los datos, creando Hipótesis que nos ayuden a tomar buenas desiciones con la información qye tenemos.

## Inicialización

In [10]:
# Cargar todas las librerías
import pandas as pd
import numpy as np
from scipy import stats as st 

## Cargar datos

In [27]:
# Carga los archivos de datos en diferentes DataFrames
df_users = pd.read_csv(r'C:\Users\10147115\Python Project Folder\TripleTen_DS\Sprint 5_Análisis estadístico de datos\Proyecto SP5\Datasets\megaline_users.csv', sep=',')
df_plans = pd.read_csv(r'C:\Users\10147115\Python Project Folder\TripleTen_DS\Sprint 5_Análisis estadístico de datos\Proyecto SP5\Datasets\megaline_plans.csv', sep=',')
df_calls = pd.read_csv(r'C:\Users\10147115\Python Project Folder\TripleTen_DS\Sprint 5_Análisis estadístico de datos\Proyecto SP5\Datasets\megaline_calls.csv', sep=',')
df_data = pd.read_csv(r'C:\Users\10147115\Python Project Folder\TripleTen_DS\Sprint 5_Análisis estadístico de datos\Proyecto SP5\Datasets\megaline_internet.csv', sep=',')
df_sms = pd.read_csv(r'C:\Users\10147115\Python Project Folder\TripleTen_DS\Sprint 5_Análisis estadístico de datos\Proyecto SP5\Datasets\megaline_messages.csv', sep=',')

## Preparar los datos

[Los datos para este proyecto se dividen en varias tablas. Explora cada una para tener una comprensión inicial de los datos. Si es necesario, haz las correcciones requeridas en cada tabla.]

## Tarifas

In [28]:
# Imprime la información general/resumida sobre el DataFrame de las tarifas
df_plans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      int64  
 4   usd_per_gb             2 non-null      int64  
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(2), int64(5), object(1)
memory usage: 260.0+ bytes


In [29]:
# Imprime una muestra de los datos para las tarifas
df_plans.head()


,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,500,20,10,0.03,0.03,surf
1,1000,30720,3000,70,7,0.01,0.01,ultimate


## Descripción

El dataframe de los planes contiene únicamente 2 filas y 8 columnas, con información sobre las tarifas que se ofrecen.

La mayoría de los datos de las columnas son de tipo números enteros o integer (int64), hay dos columnas con información monetaria (usd) que son de tipo números fraccionarios o float (float64), y finalmente una columna con información de tipo cualitativa o descriptiva (object) que son los nombres de los planes.

La información se ve bien pero podemos cambiar los tipos de datos en todas las columnas que contengan información monetaria en usd, para homologar todas las columnas con esta información.

No se tienen datos eliminados ni ausentes, los nombres de los planes están en minusculas, los nombres de las columnas cumplen el formato snake case, nombres en minúsculas, separación con guiones bajos, sin espacios al principio ni al final.

Por lo que no hay mucho que corregir en este primer dataframe revisado.

## Corregir datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

In [ ]:
# Vamos a cambiar los tipos de datos en todas las columnas con información monetaria (usd) de int64 a float64:
df_plans['usd_monthly_pay'] = df_plans['usd_monthly_pay'].astype('float64')
df_plans['usd_per_gb'] = df_plans['usd_per_gb'].astype('float64')
df_plans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      float64
 4   usd_per_gb             2 non-null      float64
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(4), int64(3), object(1)
memory usage: 260.0+ bytes


## Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

## Usuarios/as

In [33]:
# Imprime la información general/resumida sobre el DataFrame de usuarios
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   first_name  500 non-null    object
 2   last_name   500 non-null    object
 3   age         500 non-null    int64 
 4   city        500 non-null    object
 5   reg_date    500 non-null    object
 6   plan        500 non-null    object
 7   churn_date  34 non-null     object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


In [36]:
# Imprime una muestra de datos para usuarios
df_users.head()

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaN
1,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN
2,1002,Carlee,Hoffman,36,"Las Vegas-Henderson-Paradise, NV MSA",2018-10-21,surf,NaN
3,1003,Reynaldo,Jenkins,52,"Tulsa, OK MSA",2018-01-28,surf,NaN
4,1004,Leonila,Thompson,40,"Seattle-Tacoma-Bellevue, WA MSA",2018-05-23,surf,NaN


[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

### Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

### Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

## Llamadas

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de las llamadas



In [ ]:
# Imprime una muestra de datos para las llamadas



[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

### Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

### Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

## Mensajes

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de los mensajes



In [ ]:
# Imprime una muestra de datos para los mensajes



[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

### Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

### Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

## Internet

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de internet



In [ ]:
# Imprime una muestra de datos para el tráfico de internet



[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

### Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

### Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

## Estudiar las condiciones de las tarifas

[Es sumamente importante entender cómo funcionan las tarifas, cómo se les cobra a los usuarios en función de su plan de suscripción. Así que te sugerimos imprimir la información de la tarifa para ver una vez más sus condiciones.]

In [ ]:
# Imprime las condiciones de la tarifa y asegúrate de que te quedan claras



## Agregar datos por usuario

[Ahora que los datos están limpios, agrega los datos por usuario y por periodo para que solo haya un registro por usuario y por periodo. Esto facilitará mucho el análisis posterior.]

In [ ]:
# Calcula el número de llamadas hechas por cada usuario al mes. Guarda el resultado.



In [ ]:
# Calcula la cantidad de minutos usados por cada usuario al mes. Guarda el resultado.



In [ ]:
# Calcula el número de mensajes enviados por cada usuario al mes. Guarda el resultado.



In [ ]:
# Calcula el volumen del tráfico de Internet usado por cada usuario al mes. Guarda el resultado.



[Junta los datos agregados en un DataFrame para que haya un registro que represente lo que consumió un usuario único en un mes determinado.]

In [ ]:
# Fusiona los datos de llamadas, minutos, mensajes e Internet con base en user_id y month



In [ ]:
# Añade la información de la tarifa



[Calcula los ingresos mensuales por usuario (resta el límite del paquete gratuito del número total de llamadas, mensajes de texto y datos; multiplica el resultado por el valor del plan de llamadas; añade la tarifa mensual en función del plan de llamadas). Nota: Dadas las condiciones del plan, ¡esto podría no ser tan trivial como un par de líneas! Así que no pasa nada si dedicas algo de tiempo a ello.]

In [ ]:
# Calcula el ingreso mensual para cada usuario



## Estudia el comportamiento de usuario

[Calcula algunas estadísticas descriptivas para los datos agregados y fusionados que nos sean útiles y que muestren un panorama general captado por los datos. Dibuja gráficos útiles para facilitar la comprensión. Dado que la tarea principal es comparar las tarifas y decidir cuál es más rentable, las estadísticas y gráficas deben calcularse por tarifa.]

[En los comentarios hallarás pistas relevantes para las llamadas, pero no las hay para los mensajes e Internet. Sin embargo, el principio del estudio estadístico que se aplica para ellos es el mismo que para las llamadas.]

### Llamadas

In [ ]:
# Compara la duración promedio de llamadas por cada plan y por cada mes. Traza un gráfico de barras para visualizarla.



In [ ]:
# Compara el número de minutos mensuales que necesitan los usuarios de cada plan. Traza un histograma.



[Calcula la media y la variable de la duración de las llamadas para averiguar si los usuarios de los distintos planes se comportan de forma diferente al realizar sus llamadas.]

In [ ]:
# Calcula la media y la varianza de la duración mensual de llamadas.



In [ ]:
# Traza un diagrama de caja para visualizar la distribución de la duración mensual de llamadas



[Elabora las conclusiones sobre el comportamiento de los usuarios con respecto a las llamadas. ¿Su comportamiento varía en función del plan?]

### Mensajes

In [ ]:
# Comprara el número de mensajes que tienden a enviar cada mes los usuarios de cada plan



In [ ]:
# Compara la cantidad de tráfico de Internet consumido por usuarios por plan



[Elabora las conclusiones sobre el comportamiento de los usuarios con respecto a los mensajes. ¿Su comportamiento varía en función del plan?]

### Internet

[Elabora las conclusiones sobre cómo los usuarios tienden a consumir el tráfico de Internet. ¿Su comportamiento varía en función del plan?]

## Ingreso

[Del mismo modo que has estudiado el comportamiento de los usuarios, describe estadísticamente los ingresos de los planes.]

[Elabora las conclusiones sobre cómo difiere el ingreso entre los planes.]

## Prueba las hipótesis estadísticas

[Prueba la hipótesis de que son diferentes los ingresos promedio procedentes de los usuarios de los planes de llamada Ultimate y Surf.]

[Elabora las hipótesis nula y alternativa, escoge la prueba estadística, determina el valor alfa.]

In [ ]:
# Prueba las hipótesis



[Prueba la hipótesis de que el ingreso promedio de los usuarios del área NY-NJ es diferente al de los usuarios de otras regiones.]

[Elabora las hipótesis nula y alternativa, escoge la prueba estadística, determina el valor alfa.]

In [ ]:
# Prueba las hipótesis



## Conclusión general

[En esta sección final, enumera tus conclusiones importantes. Asegúrate de que estas abarquen todas las decisiones (suposiciones) importantes que adoptaste y que determinaron la forma elegida para procesar y analizar los datos.]